In [11]:
import robotic as ry
import numpy as np
from visual import getObject, lookAtObj, scanObject
from arena import *
import open3d as o3d
import random
import copy

INITIAL_OBJ_POS = [-.5, 0, .69]
ON_REAL = False
ARENA_POS = np.array([-.54, -.17, .651])

GRIPPER_FINGER_SEPARATION = .075
GRIPPER_FINGER_WIDTH = .025

def validSideDistance(p1, p2, n):
    dist = np.linalg.norm(p2-p1)
    normal_dist = abs(np.dot(p1 - p2, n) / np.linalg.norm(n))
    side_dist = np.sqrt(dist*dist-normal_dist*normal_dist)
    return side_dist <= GRIPPER_FINGER_WIDTH*.5

def validNormalDistance(p1, p2, n, dist_thresh=GRIPPER_FINGER_SEPARATION):
    return (abs(np.dot(p1 - p2, n) / np.linalg.norm(n)) <= dist_thresh
            and abs(np.dot(p1 - p2, n) / np.linalg.norm(n)) >= GRIPPER_FINGER_WIDTH)

def validNormalRelativeDir(p1, p2, n):
    p2pvec = p2-p1
    dot_product = np.dot(n, p2pvec)
    # Normal vector has magnitude one so we ignore it
    magnitude_vector = np.linalg.norm(p2pvec)
    cosine_angle = dot_product / magnitude_vector
    return cosine_angle <= 0

def validNormalDir(n1, n2):
    upvec = np.array([0., 0., 1.])
    dot_product1 = np.dot(n1, upvec)
    dot_product2 = np.dot(n2, upvec)
    return dot_product1 >= 0 and dot_product2 >= 0

def ICP_matrix(source, target, iterations=10, threshold=.02, trans_init=np.identity(4)):
    source_copy = copy.deepcopy(source)

    cumulative_transformation = np.identity(4)

    for _ in range(iterations):
        source_filtered, _ = source_copy.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
        target_filtered, _ = target.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
        source_downsampled = source_filtered.voxel_down_sample(voxel_size=0.001)
        target_downsampled = target_filtered.voxel_down_sample(voxel_size=0.001)

        reg_p2p = o3d.pipelines.registration.registration_icp(
            source_downsampled, target_downsampled, threshold, trans_init,
            o3d.pipelines.registration.TransformationEstimationPointToPoint())
        cumulative_transformation = np.dot(reg_p2p.transformation, cumulative_transformation)
        source_copy.transform(reg_p2p.transformation)

    return cumulative_transformation

def mstorePCR(point_clouds, smoothing=False, verbose=0):
    result = point_clouds[0]
    
    for i in range(len(point_clouds)-1):
        source, target = point_clouds[i+1], point_clouds[i]
        m = ICP_matrix(source, target)

        source.transform(m)
        result += source

        if smoothing:
            result, _ = result.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
            result = result.voxel_down_sample(voxel_size=0.001)

    if verbose:
        o3d.visualization.draw_geometries([result])
    return result

In [12]:
#-- load parameters, typically automatically from 'rai.cfg'
ry.params_print()
verbose = 0

#-- define a configuration
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))

obj = C.addFrame('obj')
obj.setShape(ry.ST.ssBox, size=[.05, .09, .1, .005])
obj.setPosition(INITIAL_OBJ_POS)
obj.setColor([1, .5, .5])
obj.setMass(.1)
obj.setContact(True)

C.addFrame('predicted_obj') \
    .setShape(ry.ST.marker, size=[.1]) \
    .setPosition(INITIAL_OBJ_POS) \
    .setColor([1, 0, 0])

for i in range(32):
    C.addFrame(f'view_point_{i}') \
        .setPosition([0, 0, 0]) \
        .setShape(ry.ST.sphere, size=[.02]) \
        .setColor([1, 1, 0])

bot = ry.BotOp(C, ON_REAL)

obj_pos = INITIAL_OBJ_POS

# Generate Arena
arena = CircularArena(middleP=ARENA_POS, innerR=None, outerR=.3)
arena.plotArena(C)

-- module.cpp:operator():95(0) python,
RealSense/resolution: 640,
RealSense/alignToDepth!,
physx/motorKp: 10000,
physx/motorKd: 1000,
physx/multiBody!,
bot/useGripper,
bot/useRobotiq!,
bot/useArm: left,
bot/blockRealRobot!,
botsim/hyperSpeed: 1,
botsim/verbose: 1,
botsim/engine: physx,
physx/verbose: 1,
physx/yGravity!,
physx/softBody!,
physx/multiBodyDisableGravity,
physx/jointedBodies!,
physx/angularDamping: 0.1,
physx/defaultFriction: 1,
physx/defaultRestitution: 0.1,
bot/useOptitrack!,
bot/useAudio!,
KOMO/verbose: 1,
KOMO/animateOptimization: 0,
KOMO/mimicStable,
KOMO/useFCL,
KOMO/unscaleEqIneqReport!,
KOMO/sampleRate_stable: 0,
opt/verbose: 1,
opt/stopTolerance: 0.01,
opt/stopFTolerance: -1,
opt/stopGTolerance: -1,
opt/stopEvals: 1000,
opt/stopIters: 1000,
opt/stopOuters: 1000,
opt/stopLineSteps: 10,
opt/stopTinySteps: 10,
opt/initStep: 1,
opt/minStep: -1,
opt/maxStep: 0.2,
opt/damping: 1,
opt/stepInc: 1.5,
opt/stepDec: 0.5,
opt/wolfe: 0.01,
opt/boundedNewton,
opt/muInit: 1,
opt/m

In [13]:
# Point towards set initial object position
lookAtObj(bot, C, np.array(INITIAL_OBJ_POS))

# Capture midpoint from point cloud
obj_pos  = getObject(bot, C, arena) 

scanned_views_ = scanObject(bot, C, np.array(obj_pos), arena, view_count=8)
scanned_views = []
for sv in scanned_views_:
    point_cloud_obj = o3d.geometry.PointCloud()
    point_cloud_obj.points = o3d.utility.Vector3dVector(sv)
    scanned_views.append(point_cloud_obj)

{ time: 0.002, evals: 9, done: 1, feasible: 1, sos: 0.0102726, f: 0, ineq: 0, eq: 0.0112763 }
-- WARNING:kin.cpp:getFrame:393(-1) cannot find frame named 'pcl'
{ time: 0.005571, evals: 19, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.00303904 }
{ time: 0.001321, evals: 12, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.00422288 }
{ time: 0.001842, evals: 12, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.00388187 }
{ time: 0.018514, evals: 72, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.072641 }
{ time: 0.003715, evals: 11, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.00361921 }
{ time: 0.00183, evals: 20, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.00308296 }
{ time: 0.043111, evals: 372, done: 1, feasible: 0, sos: 0, f: 0, ineq: 0, eq: 0.82426 }
{ time: 0.013538, evals: 57, done: 1, feasible: 0, sos: 0, f: 0, ineq: 0, eq: 0.821359 }


In [14]:
final_points = mstorePCR(scanned_views)

Starting loop...
Ending loop...
Starting loop...
Ending loop...


In [15]:
pclFrame = C.getFrame("pcl")
pclFrame.setPointCloud(np.asarray(final_points.points))
C.view_recopyMeshes()

In [16]:
def findPossibleGrasps(point_cloud, voxel_size=.005, verbose=3):

    # Prepare point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)
    if voxel_size: pcd = pcd.voxel_down_sample(voxel_size=voxel_size)
    pcd.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    if verbose > 2:
        o3d.visualization.draw_geometries([pcd], point_show_normal=True)
    
    # Find antipodal pairs
    antipodal_pairs = []
    for i in range(len(pcd.points)):
        normal_i = np.asarray(pcd.normals[i])
        for j in range(i + 1, len(pcd.points)):
            normal_j = np.asarray(pcd.normals[j])
            dot_product = np.dot(normal_i, normal_j)
            if (dot_product < -0.7 and
                validSideDistance(pcd.points[i], pcd.points[j], pcd.normals[i]) and
                validNormalDistance(pcd.points[i], pcd.points[j], pcd.normals[i]) and
                validNormalRelativeDir(pcd.points[i], pcd.points[j], pcd.normals[i])
                and validNormalDir(pcd.points[j], pcd.normals[i])
                ):
                    antipodal_pairs.append((i, j))
        if verbose > 2 and (i+1)%100 == 0:
            print(f"Matching atimodal points: {i+1}/{len(pcd.points)} ({((i+1)*100/len(pcd.points)):.2f}%) Done")

    if verbose:
        print("Antipodal pairs found:", antipodal_pairs)

        if verbose > 1:
            # Create a line set to visualize antipodal pairs
            line_set = o3d.geometry.LineSet()

            # Populate line_set with antipodal pairs
            lines = []
            for pair in antipodal_pairs:
                lines.append([pair[0], pair[1]])
            lines = np.array(lines).astype(np.int32)
            line_set.points = o3d.utility.Vector3dVector(np.asarray(pcd.points))
            line_set.lines = o3d.utility.Vector2iVector(lines)

            # Set color of the lines to red
            line_colors = np.array([[1, 0, 0] for _ in range(len(lines))])
            line_set.colors = o3d.utility.Vector3dVector(line_colors)

            # Visualize the point cloud and antipodal pairs
            o3d.visualization.draw_geometries([pcd, line_set], point_show_normal=True)

    return antipodal_pairs

point_indices = findPossibleGrasps(np.asarray(final_points.points), voxel_size=.005, verbose=3)

Matching atimodal points: 100/2149 (4.65%) Done
Matching atimodal points: 200/2149 (9.31%) Done
Matching atimodal points: 300/2149 (13.96%) Done
Matching atimodal points: 400/2149 (18.61%) Done
Matching atimodal points: 500/2149 (23.27%) Done
Matching atimodal points: 600/2149 (27.92%) Done
Matching atimodal points: 700/2149 (32.57%) Done
Matching atimodal points: 800/2149 (37.23%) Done
Matching atimodal points: 900/2149 (41.88%) Done
Matching atimodal points: 1000/2149 (46.53%) Done
Matching atimodal points: 1100/2149 (51.19%) Done
Matching atimodal points: 1200/2149 (55.84%) Done
Matching atimodal points: 1300/2149 (60.49%) Done
Matching atimodal points: 1400/2149 (65.15%) Done
Matching atimodal points: 1500/2149 (69.80%) Done
Matching atimodal points: 1600/2149 (74.45%) Done
Matching atimodal points: 1700/2149 (79.11%) Done
Matching atimodal points: 1800/2149 (83.76%) Done
Matching atimodal points: 1900/2149 (88.41%) Done
Matching atimodal points: 2000/2149 (93.07%) Done
Matching at

In [17]:
random_grasp_indices = random.choice(point_indices)
gripping_pos = (final_points.points[random_grasp_indices[0]] + final_points.points[random_grasp_indices[1]]) * .5
gripping_angle = final_points.points[random_grasp_indices[0]] + final_points.points[random_grasp_indices[1]]
gripping_angle /= np.linalg.norm(gripping_angle)

In [18]:
bot.home(C)
while bot.getTimeToEnd() > 0:
    bot.sync(C, .1)
    
bot.gripperMove(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

bot.sync(C)

komo = ry.KOMO()
komo.setConfig(C, True)
komo.setTiming(1., 1, 1., 0)

komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)

komo.addObjective([1.], ry.FS.position, ['l_gripper'], ry.OT.eq, [1e1], gripping_pos)
komo.addObjective([1.], ry.FS.vectorX, ["l_gripper"], ry.OT.eq, [1e1], gripping_angle)

ret = ry.NLP_Solver().setProblem(komo.nlp()).setOptions(stopTolerance=1e-2, verbose=0).solve()
print(ret)
bot.move(komo.getPath(), [3.])
while bot.getTimeToEnd() > 0:
    bot.sync(C, .1)

bot.gripperClose(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

bot.home(C)
while bot.getTimeToEnd() > 0:
    bot.sync(C, .1)

bot.gripperMove(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)

{ time: 0.00465, evals: 19, done: 1, feasible: 1, sos: 0, f: 0, ineq: 0, eq: 0.122147 }


In [19]:
bot.home(C)
C.view(True)

0

In [20]:
del bot
del C

-- bot.cpp:~BotOp:118(0) shutting down BotOp...
-- simulation.cpp:~BotThreadedSim:56(0) shutting down SimThread
-- simulation.cpp:~Simulation:148(0) shutting down Simulation
